<a href="https://colab.research.google.com/github/pullz6/Research_inspired_Self-Explaining-Neural-Networks-for-Business-Process-Monitoring/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libraries

In [136]:
#Importing libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import datetime
from sklearn.preprocessing import MinMaxScaler

In [137]:
df = pd.read_csv("Helpdesk.csv")

In [138]:
df.head()

,concept:name,lifecycle:transition,org:resource,time:timestamp,Activity,Resource,case:concept:name,case:variant,case:variant-index,case:creator
0,Assign seriousness,complete,Value 2,2010-01-13 08:40:25+02:00,Assign seriousness,Value 2,Case3608,Variant 33,33,Fluxicon Disco
1,Take in charge ticket,complete,Value 2,2010-01-29 08:52:27+02:00,Take in charge ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
2,Resolve ticket,complete,Value 2,2010-01-29 08:52:34+02:00,Resolve ticket,Value 2,Case3608,Variant 33,33,Fluxicon Disco
3,Closed,complete,Value 5,2010-02-13 08:52:48+02:00,Closed,Value 5,Case3608,Variant 33,33,Fluxicon Disco
4,Closed,complete,Value 5,2010-02-13 08:52:48+02:00,Closed,Value 5,Case3608,Variant 33,33,Fluxicon Disco


In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21348 entries, 0 to 21347
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   concept:name          21348 non-null  object
 1   lifecycle:transition  21348 non-null  object
 2   org:resource          21348 non-null  object
 3   time:timestamp        21348 non-null  object
 4   Activity              21348 non-null  object
 5   Resource              21348 non-null  object
 6   case:concept:name     21348 non-null  object
 7   case:variant          21348 non-null  object
 8   case:variant-index    21348 non-null  int64 
 9   case:creator          21348 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.6+ MB


In [140]:
df.drop_duplicates(inplace=True)

In [141]:
df['time:timestamp'] = pd.to_datetime(df['time:timestamp'], format='mixed')

<ipython-input-141-ac090aac2b9b>:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['time:timestamp'] = pd.to_datetime(df['time:timestamp'], format='mixed')


In [142]:
df['new_date'] = [d.date() for d in df['time:timestamp']]
df['new_time'] = [d.time() for d in df['time:timestamp']]

In [143]:
min_timestamps = df.groupby('case:concept:name')['time:timestamp'].transform('min')

In [144]:
df['time_lapsed'] = df['time:timestamp'] - min_timestamps

In [145]:
df['time_lapsed'] = pd.to_timedelta(df['time_lapsed'])

In [146]:
df['time_lapsed'] = df['time_lapsed'].dt.total_seconds()

In [147]:
df['concept:name'].unique()

array(['Assign seriousness', 'Take in charge ticket', 'Resolve ticket',
       'Closed', 'Wait', 'Create SW anomaly', 'Insert ticket',
       'Schedule intervention', 'RESOLVED', 'INVALID', 'VERIFIED',
       'Resolve SW anomaly', 'Require upgrade', 'DUPLICATE'], dtype=object)

In [148]:
#Checking if we can concept:name and activity is the same as well as the org:resource and the resource, so that we can delete this feild.
df['Checker_1'] = (df['concept:name'] == df['Activity'])
df['Checker_2'] = (df['org:resource'] == df['Resource'])

In [149]:
#Let's see if it all the same, if True.
df['Checker_2'].unique()

array([ True])

In [150]:
#Lets drop it since it is the same
df.drop(['concept:name','org:resource','Checker_1','Checker_2','case:creator','lifecycle:transition','case:variant'],axis=1, inplace=True)

In [151]:
#Lets create a stage number for each activity stage of the help desk, this will help us easily identify where the case is in terms of the process

#This is also to immulate the label encoder
#First create a dataframe with stage numbers and the stage names
stage = pd.DataFrame()
stage['Stage_no'] = [1,2,3,4,5,6,7,8,9,9.1,9.2,10]
stage['Activity'] = ['Insert ticket','Assign seriousness','Take in charge ticket','Wait','Require upgrade','Schedule intervention','Resolve ticket','VERIFIED','RESOLVED','INVALID', 'DUPLICATE','Closed']

In [152]:
df=pd.merge(df,stage,on='Activity', how='left')

In [153]:
df.head()

,time:timestamp,Activity,Resource,case:concept:name,case:variant-index,new_date,new_time,time_lapsed,Stage_no
0,2010-01-13 08:40:25+02:00,Assign seriousness,Value 2,Case3608,33,2010-01-13,08:40:25,0.0,2.0
1,2010-01-29 08:52:27+02:00,Take in charge ticket,Value 2,Case3608,33,2010-01-29,08:52:27,1383122.0,3.0
2,2010-01-29 08:52:34+02:00,Resolve ticket,Value 2,Case3608,33,2010-01-29,08:52:34,1383129.0,7.0
3,2010-02-13 08:52:48+02:00,Closed,Value 5,Case3608,33,2010-02-13,08:52:48,2679143.0,10.0
4,2010-01-13 12:26:04+02:00,Assign seriousness,Value 2,Case2748,1,2010-01-13,12:26:04,0.0,2.0


In [154]:
df.describe()

,case:variant-index,time_lapsed,Stage_no
count,21221.000000,2.122100e+04,21142.000000
mean,14.502427,1.167490e+06,5.274544
std,35.928597,1.483251e+06,3.076202
min,1.000000,0.000000e+00,1.000000
25%,1.000000,2.400000e+01,3.000000
50%,2.000000,2.707180e+05,4.000000
75%,7.000000,2.519608e+06,7.000000
max,226.000000,5.183565e+06,10.000000


In [156]:
df.head()

,time:timestamp,Activity,Resource,case:concept:name,case:variant-index,new_date,new_time,time_lapsed,Stage_no
0,2010-01-13 08:40:25+02:00,Assign seriousness,Value 2,Case3608,33,2010-01-13,08:40:25,0.0,2.0
1,2010-01-29 08:52:27+02:00,Take in charge ticket,Value 2,Case3608,33,2010-01-29,08:52:27,1383122.0,3.0
2,2010-01-29 08:52:34+02:00,Resolve ticket,Value 2,Case3608,33,2010-01-29,08:52:34,1383129.0,7.0
3,2010-02-13 08:52:48+02:00,Closed,Value 5,Case3608,33,2010-02-13,08:52:48,2679143.0,10.0
4,2010-01-13 12:26:04+02:00,Assign seriousness,Value 2,Case2748,1,2010-01-13,12:26:04,0.0,2.0


In [158]:
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [159]:
X_sequences = []
y_act = []
y_time = []

In [160]:
case_groups = df.groupby('case:concept:name')
max_len = 14  # As per your model's expected timesteps
num_features = 26  # Must match your feature count

In [161]:
import numpy as np
a = np.array((7,2,3))
b = np.array((2,9,4))
np.column_stack((a,b))

array([[7, 2],
       [2, 9],
       [3, 4]])

In [162]:
for _, group in case_groups:
    group = group.sort_values('time:timestamp')

    # Create feature matrix (ensure no NaT/NaN)
    try:
        features = np.column_stack([
            group['Activity'].values,
            group['time_lapsed'].astype('float32').values.reshape(-1, 1),
            group['new_date'].dt.dayofweek.values / 6,
            group['new_date'].dt.hour.values / 23
        ])
    except Exception as e:
        print(f"Error in group {_}: {e}")
        continue  # Skip problematic groups

    # Pad sequences (handle empty cases)
    if features.size == 0:
        padded = np.zeros((max_len, n_features), dtype='float32')  # Define n_features
    else:
        padded = pad_sequences(
            [features],
            maxlen=max_len,
            padding='post',
            truncating='post',
            dtype='float32'
        )[0]

    X_sequences.append(padded)
    y_act.append(pd.get_dummies(group['Stage_no']).values[-1])
    y_time.append(group['time_lapsed'].values[-1])

Error in group Case1: Can only use .dt accessor with datetimelike values
Error in group Case10: Can only use .dt accessor with datetimelike values
Error in group Case100: Can only use .dt accessor with datetimelike values
Error in group Case1000: Can only use .dt accessor with datetimelike values
Error in group Case1001: Can only use .dt accessor with datetimelike values
Error in group Case1002: Can only use .dt accessor with datetimelike values
Error in group Case1003: Can only use .dt accessor with datetimelike values
Error in group Case1004: Can only use .dt accessor with datetimelike values
Error in group Case1005: Can only use .dt accessor with datetimelike values
Error in group Case1006: Can only use .dt accessor with datetimelike values
Error in group Case1007: Can only use .dt accessor with datetimelike values
Error in group Case1008: Can only use .dt accessor with datetimelike values
Error in group Case1009: Can only use .dt accessor with datetimelike values
Error in group Cas

Model building

In [115]:
!pip install keras

In [116]:
#Creating the model architecture

In [117]:
#Importing the required libriares since the LSTM model will be done with Pytorch
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.layers import LSTM, GRU, SimpleRNN
from keras.layers import Input
from keras.utils import get_file
from keras.optimizers import Nadam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import BatchNormalization

In [118]:
main_input = Input(shape=(max_len, num_features), name='main_input')

In [119]:
main_input

<KerasTensor shape=(None, 15, 26), dtype=float32, sparse=False, name=main_input>

In [120]:
input_data = df[['new_date','new_time','time_lapsed','Activity_Assign seriousness', 'Activity_Closed',
       'Activity_Create SW anomaly', 'Activity_DUPLICATE', 'Activity_INVALID',
       'Activity_Insert ticket', 'Activity_RESOLVED',
       'Activity_Require upgrade', 'Activity_Resolve SW anomaly',
       'Activity_Resolve ticket', 'Activity_Schedule intervention',
       'Activity_Take in charge ticket', 'Activity_VERIFIED', 'Activity_Wait']]

In [121]:
#Create the training split

In [122]:
l1 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=True, dropout=0.2)(main_input) # the shared layer
b1 = BatchNormalization()(l1)
l2_1 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=False, dropout=0.2)(b1) # the layer specialized in activity prediction
b2_1 = BatchNormalization()(l2_1)
l2_2 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=False, dropout=0.2)(b1) # the layer specialized in time prediction
b2_2 = BatchNormalization()(l2_2)
act_output = Dense(len(df['Stage_no'].unique()), activation='softmax', kernel_initializer='glorot_uniform', name='act_output')(b2_1)
time_output = Dense(1, kernel_initializer='glorot_uniform', name='time_output')(b2_2)

In [123]:
model = Model(inputs=[main_input], outputs=[act_output, time_output])

opt = Nadam(learning_rate=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, weight_decay=0.004, clipvalue=3)

model.compile(loss={'act_output':'categorical_crossentropy', 'time_output':'mae'}, optimizer=opt)
early_stopping = EarlyStopping(monitor='val_loss', patience=42)
model_checkpoint = ModelCheckpoint('output_files/models/model_{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

In [124]:
model.fit(X_sequences, {'act_output':y_act, 'time_output':y_time}, validation_split=0.2, verbose=2, callbacks=[early_stopping, model_checkpoint, lr_reducer], batch_size=max_len, epochs=500)

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed